In [2]:
import pandas as pd
import os
import time
from datetime import datetime
from time import mktime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
style.use("dark_background")
import re

In [2]:
path = 'C:/Users/USER/intraQuarter'

def key_stats(gather=[  'Total Debt/Equity',
                        'Trailing P/E',
                        'Price/Sales',
                        'Price/Book',
                        'Profit Margin',
                        'Operating Margin',
                        'Return on Assets',
                        'Return on Equity',
                        'Revenue Per Share',
                        'Market Cap',
                        'Enterprise Value',
                        'Forward P/E',
                        'PEG Ratio',
                        'Enterprise Value/Revenue',
                        'Enterprise Value/EBITDA',
                        'Revenue',
                        'Gross Profit',
                        'EBITDA',
                        'Net Income Avl to Common ',
                        'Diluted EPS',
                        'Earnings Growth',
                        'Revenue Growth',
                        'Total Cash',
                        'Total Cash Per Share',
                        'Total Debt',
                        'Current Ratio',
                        'Book Value Per Share',
                        'Cash Flow',
                        'Beta',
                        'Held by Insiders',
                        'Held by Institutions',
                        'Shares Short (as of',
                        'Short Ratio',
                        'Short % of Float',
                        'Shares Short (prior ']):
    
    stats_path = path + '/_KeyStats'
    stock_list = [x[0] for x in os.walk(stats_path)]
    
    df = pd.DataFrame(columns = ['Date',
                                 'Unix',
                                 'Ticker',
                                 'Price',
                                 'stock_p_change',
                                 'SP500',
                                 'sp500_p_change',
                                 'Difference',
                                 'DE Ratio',
                                 'Trailing P/E',
                                 'Price/Sales',
                                 'Price/Book',
                                 'Profit Margin',
                                 'Operating Margin',
                                 'Return on Assets',
                                 'Return on Equity',
                                 'Revenue Per Share',
                                 'Market Cap',
                                 'Enterprise Value',
                                 'Forward P/E',
                                 'PEG Ratio',
                                 'Enterprise Value/Revenue',
                                 'Enterprise Value/EBITDA',
                                 'Revenue',
                                 'Gross Profit',
                                 'EBITDA',
                                 'Net Income Avl to Common ',
                                 'Diluted EPS',
                                 'Earnings Growth',
                                 'Revenue Growth',
                                 'Total Cash',
                                 'Total Cash Per Share',
                                 'Total Debt',
                                 'Current Ratio',
                                 'Book Value Per Share',
                                 'Cash Flow',
                                 'Beta',
                                 'Held by Insiders',
                                 'Held by Institutions',
                                 'Shares Short (as of',
                                 'Short Ratio',
                                 'Short % of Float',
                                 'Shares Short (prior ',                                
                                 'Status'])
    
    sp500_df = pd.DataFrame.from_csv('s&p500.csv')
    sp500_df = sp500_df.reindex(columns = ['Open','High','Low','Close','Volume','Adj Close'])
    stock_df = pd.DataFrame.from_csv("stock_prices(1).csv")
    
    ticker_list = []
    
    for each_dir in stock_list[1:]:
        each_file = os.listdir(each_dir)
        ticker = each_dir.split('\\')[1]
        ticker_list.append(ticker)
        
        
        if len(each_file) > 0:
            
            for file in each_file:
                date_stamp = datetime.strptime(file, "%Y%m%d%H%M%S.html")
                unix_time = time.mktime(date_stamp.timetuple())
                full_file_path = each_dir + '\\' + file
                source = open(full_file_path, 'r').read()
                try:
                    value_list = []
                    for each_data in gather:                   

                        try:
                            #value = float(source.split(gather + ':</td><td class="yfnc_tabledata1">')[1].split('</td>')[0])
                            regex = re.escape(each_data) + r'.*?(\d{1,8}\.\d{1,8}M?B?|N/A)%?</td>'
                            value = re.search(regex, source)
                            value = (value.group(1))
                        
                            if 'B' in value:
                                value = float(value.replace("B", ""))*1000000000
                            elif 'M' in value:
                                value = float(value.replace("M", ""))*1000000
                                
                            value_list.append(value)    

                        except Exception as e:
                            value = 'N/A'
                            value_list.append(value)
                          
                    
                    try:
                        sp500_date = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d')
                        row = sp500_df[(sp500_df.index == sp500_date)]
                        sp500_value = float(row['Adj Close'])
                    
                    except:
                        sp500_date = datetime.fromtimestamp(unix_time - 259200).strftime('%Y-%m-%d')
                        row = sp500_df[(sp500_df.index == sp500_date)]
                        sp500_value = float(row['Adj Close'])
                        
                        
                    one_year_later = int(unix_time + 31536000)    
                        
                        
                    try:
                        sp500_1y = datetime.fromtimestamp(one_year_later).strftime('%Y-%m-%d')
                        row = sp500_df[(sp500_df.index == sp500_1y)]
                        sp500_1y_value = float(row["Adj Close"])
                    except:
                        try:
                            sp500_1y = datetime.fromtimestamp(one_year_later-259200).strftime('%Y-%m-%d')
                            row = sp500_df[(sp500_df.index == sp500_1y)]
                            sp500_1y_value = float(row["Adj Close"])
                        except Exception as e:
                            pass             
                            
                    try:
                        stock_price_1y = datetime.fromtimestamp(one_year_later).strftime('%Y-%m-%d')
                        row = stock_df[(stock_df.index == stock_price_1y)][ticker.upper()]
                        

                        stock_1y_value = round(float(row),2)
                    
                    except Exception as e:
                        try:
                            stock_price_1y = datetime.fromtimestamp(one_year_later-259200).strftime('%Y-%m-%d')
                            row = stock_df[(stock_df.index == stock_price_1y)][ticker.upper()]
                            stock_1y_value = round(float(row),2)
                        except Exception as e:
                            pass                          
                    
                    try:
                        stock_price = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d')
                        row = stock_df[(stock_df.index == stock_price)][ticker.upper()]
                        stock_price = round(float(row),2)
                    
                    except Exception as e:
                        try:
                            stock_price = datetime.fromtimestamp(unix_time-259200).strftime('%Y-%m-%d')
                            row = stock_df[(stock_df.index == stock_price)][ticker.upper()]
                            stock_price = round(float(row),2)
                        except Exception as e:
                            pass
                    
                    stock_p_change = round(( ((stock_1y_value - stock_price) / stock_price) * 100 ), 2)
                    sp500_p_change = round(( ((sp500_1y_value - sp500_value) / sp500_value) * 100 ), 2)
                    difference = stock_p_change - sp500_p_change
                    
                    if difference > 5:
                        status = 1
                    else:
                        status = 0
                        
                    if value_list.count('N/A') > 15:
                        pass
                    else:
                        df = df.append({'Date':date_stamp,
                                            'Unix':unix_time,
                                            'Ticker':ticker,
                                            
                                            'Price':stock_price,
                                            'stock_p_change':stock_p_change,
                                            'SP500':sp500_value,
                                            'sp500_p_change':sp500_p_change,
                                            'Difference':difference,
                                            'DE Ratio':value_list[0],
                                            'Trailing P/E':value_list[1],
                                            'Price/Sales':value_list[2],
                                            'Price/Book':value_list[3],
                                            'Profit Margin':value_list[4],
                                            'Operating Margin':value_list[5],
                                            'Return on Assets':value_list[6],
                                            'Return on Equity':value_list[7],
                                            'Revenue Per Share':value_list[8],
                                            'Market Cap':value_list[9],
                                             'Enterprise Value':value_list[10],
                                             'Forward P/E':value_list[11],
                                             'PEG Ratio':value_list[12],
                                             'Enterprise Value/Revenue':value_list[13],
                                             'Enterprise Value/EBITDA':value_list[14],
                                             'Revenue':value_list[15],
                                             'Gross Profit':value_list[16],
                                             'EBITDA':value_list[17],
                                             'Net Income Avl to Common ':value_list[18],
                                             'Diluted EPS':value_list[19],
                                             'Earnings Growth':value_list[20],
                                             'Revenue Growth':value_list[21],
                                             'Total Cash':value_list[22],
                                             'Total Cash Per Share':value_list[23],
                                             'Total Debt':value_list[24],
                                             'Current Ratio':value_list[25],
                                             'Book Value Per Share':value_list[26],
                                             'Cash Flow':value_list[27],
                                             'Beta':value_list[28],
                                             'Held by Insiders':value_list[29],
                                             'Held by Institutions':value_list[30],
                                             'Shares Short (as of':value_list[31],
                                             'Short Ratio':value_list[32],
                                             'Short % of Float':value_list[33],
                                             'Shares Short (prior ':value_list[34],
                                            'Status':status},
                                           ignore_index=True)

                except Exception as e:
                    pass
                
    
    df.to_csv("Key_Stats_acc_perf_WITH_NA_enhanced.csv")    

In [5]:
dfa = pd.read_csv("key_stats(1).csv")
dfa.head(100)

,Unnamed: 0,Date,Unix,Ticker,Price,stock_p_change,SP500,sp500_p_change,Difference,DE Ratio,...,Book Value Per Share,Cash Flow,Beta,Held by Insiders,Held by Institutions,Shares Short (as of,Short Ratio,Short % of Float,Shares Short (prior,Status
0,0,2011-08-02 06:45:37,1.312293e+09,a,40.79,11.296044,1254.05,10.867009,0.429035,54.360,...,11.360000,1.385000e+10,1.550,0.25,82.10,4360000.0,1.200,1.40,5420000.0,outperform
1,1,2013-05-14 00:28:58,1.368517e+09,a,43.04,17.435198,1650.34,45.901886,-28.466688,44.120,...,15.410000,1.320000e+09,1.660,0.23,83.20,4370000.0,1.200,1.30,3080000.0,underperform
2,2,2013-09-06 06:09:52,1.378473e+09,a,47.68,30.095498,1655.17,46.328892,-16.233394,56.390,...,14.460000,1.260000e+09,1.810,0.23,83.50,2780000.0,1.200,0.80,3300000.0,underperform
3,3,2004-04-13 04:20:10,1.081855e+09,aa,34.26,0.000000,1129.44,0.000000,0.000000,0.595,...,14.078000,2.530000e+09,1.808,1.00,79.36,12540000.0,2.497,1.46,12700000.0,underperform
4,4,2004-09-09 16:15:45,1.094772e+09,aa,32.89,-3.998832,1118.38,-0.979246,-3.019586,0.561,...,14.049000,1.780000e+09,1.676,1.00,76.64,7780000.0,1.935,0.90,7930000.0,underperform
5,5,2004-11-24 10:37:20,1.101321e+09,aa,33.58,-1.984822,1181.76,4.632384,-6.617206,0.530,...,14.360000,2.140000e+09,1.736,1.04,77.19,9910000.0,2.209,1.15,10140000.0,underperform
6,6,2004-12-05 00:36:42,1.102236e+09,aa,32.86,-4.086398,1190.33,5.391167,-9.477565,0.530,...,14.360000,2.140000e+09,1.736,1.04,77.11,9910000.0,2.209,1.15,10140000.0,underperform
7,7,2005-01-13 07:18:32,1.105630e+09,aa,29.43,-14.098074,1177.45,4.250779,-18.348853,0.457,...,15.207000,2.140000e+09,1.611,1.04,77.16,12810000.0,2.918,1.49,9910000.0,underperform
8,8,2005-02-07 03:48:37,1.107777e+09,aa,29.37,-14.273205,1201.72,6.399632,-20.672837,0.457,...,15.207000,2.140000e+09,1.611,1.04,75.67,11840000.0,1.892,1.37,12810000.0,underperform
9,9,2011-09-03 09:21:23,1.315067e+09,aa,12.04,-64.856976,1218.89,7.919854,-72.776830,48.260,...,14.660000,2.095000e+10,2.280,0.09,59.60,53730000.0,1.900,5.10,59140000.0,underperform
